In [46]:
import os
import json
import pandas as pd
import traceback

In [47]:
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
from langchain.chat_models import ChatOpenAI

In [49]:
baseUrl = "http://localhost:3040/v1/"

In [50]:
KEY=os.getenv('OPENAI_API_KEY')

In [51]:
llm=ChatOpenAI(openai_api_key=KEY,model='gpt-3.5-turbo', temperature=0.5, base_url=baseUrl)

In [52]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000018B025A4D00>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000018B0254D850>, temperature=0.5, openai_api_key='sk-None-GKYBmwz0kKoL5gU0syIZT3BlbkFJDTYqmsSta3Mq2RtU9byK', openai_api_base='http://localhost:3040/v1/', openai_proxy='')

In [53]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [54]:
RESPONSE_JSON = {
  "1": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer",
  },
  "2": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer",
  },
  "3": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer",
  },
  "4": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer",
  },

}

In [55]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [56]:
quiz_generation_prompt = PromptTemplate(
  input_variables=["text", "number", "subject", "tone", "response_json"],
  template=TEMPLATE
)

In [57]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [58]:
TEMPLATE2="""
You are an expert grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at per the congnitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [59]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [60]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [61]:
generate_evaluation_chain=SequentialChain(
  chains=[quiz_chain, review_chain], 
  input_variables=["text", "number", "subject", "tone", "response_json"],
  output_variables=["quiz", "review"], verbose=True
  )

In [62]:
file_path=r"D:\mcqgen\data.txt"

In [63]:
with open(file_path, 'r') as file:
  TEXT=file.read()

In [64]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [65]:
#Serialize the Python dictionary into a JSON-formatted_string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [66]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="Simple"

In [67]:
#Setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
  response=generate_evaluation_chain(
    {
      "text": TEXT,
      "number": NUMBER,
      "subject": SUBJECT,
      "tone": TONE,
      "response_json": json.dumps(RESPONSE_JSON)
    }
  )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

TypeError: 'NoneType' object is not iterable